In [1]:
import numpy as np
import networkx as nx
import random

In [2]:
def evaluate(tree):
    return sum(1 for node in tree.nodes if tree.degree(node) == 1)


In [3]:
def construct_solution(G, alpha = 1, beta = 2):
    tree = nx.Graph()
    nodes = list(G.nodes)
    random.shuffle(nodes)
    tree.add_node(nodes[0])

    degree = dict(G.degree())
    
    while len(tree.nodes) < len(nodes):
        candidates = [(u, v) for u in tree.nodes for v in G.neighbors(u) if v not in tree.nodes]
        if not candidates:
            break
        
        weights = [G[u][v]['pheromone'] ** alpha * degree[v] ** beta for u, v in candidates]
        chosen_edge = random.choices(candidates, weights=weights)[0]
        tree.add_edge(*chosen_edge)
    
    return tree

In [4]:
def update_pheromone(G, ant_solutions, evaporation_rate):
    for u, v in G.edges:
        G[u][v]['pheromone'] *= (1 - evaporation_rate)

    total_score = sum(score for _, score in ant_solutions)
    for tree, score in ant_solutions:
        for u, v in tree.edges:
            G[u][v]['pheromone'] += (score/total_score)

In [5]:
def aco(G, num_ants = 10, num_iters = 50, alpha = 1, beta = 2, evaporation_rate = 0.5):
    best_tree = None
    best_score = 0
    for _ in range(num_iters):
        solutions = [construct_solution(G, alpha, beta) for _ in range(num_ants)]
        evaluations = [evaluate(solutions[i]) for i in range(num_ants)]
        ant_solutions = list(zip(solutions, evaluations))
        best_ant = max(ant_solutions, key=lambda x: x[1])
        if best_ant[1] > best_score:
            best_tree, best_score = best_ant
        update_pheromone(G, ant_solutions, evaporation_rate)
        
    return best_tree